### 03 - HEAT STRESS OUTPUTS EXPLORATION

#### 03.1 - Libraries import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 03.2 - Temporal patterns analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ========== PARAMETRI MODIFICABILI ==========
csv_path = "heatstress_all_timestamps_all_years.csv"
metodo_statistico = "95° Perc. (°C)"  # Usa esattamente i nomi nel CSV
min_soglie_superate = 3  # Minimo numero di indici che devono superare la soglia

# ========== CARICAMENTO ==========
df = pd.read_csv(csv_path, parse_dates=["Timestamp"])

# Aggiungi colonna "Anno"
df["Anno"] = df["Timestamp"].dt.year

# ========== VALIDAZIONE NOMI COLONNE ==========
col_valore = metodo_statistico
col_confronto = f"{metodo_statistico.split(' (')[0]} > Soglia"
col_soglia = "Soglia (°C)"

# Controlla che le colonne esistano
for col in [col_valore, col_confronto, col_soglia]:
    if col not in df.columns:
        raise ValueError(f"Colonna '{col}' non trovata. Colonne disponibili: {df.columns.tolist()}")

# ========== COSTRUZIONE GIORNI PERICOLOSI ==========
giorni_pericolo = []

# Raggruppa per giorno (Timestamp)
for giorno, gruppo in df.groupby("Timestamp"):
    gruppo = gruppo.set_index("Indice")
    n_superati = (gruppo[col_confronto] == "si").sum()

    if n_superati >= min_soglie_superate:
        residui = (gruppo.loc[gruppo[col_confronto] == "si", col_valore] -
                   gruppo.loc[gruppo[col_confronto] == "si", col_soglia]).sum()

        giorni_pericolo.append({
            "Data": giorno,
            "Anno": giorno.year,
            "n_superata": n_superati,
            "residuo_totale": residui
        })

# ========== AGGREGAZIONE PER ANNO ==========
df_pericolo = pd.DataFrame(giorni_pericolo)
anni_completi = sorted(df["Anno"].unique())

summary = df_pericolo.groupby("Anno").agg({
    "Data": "count",
    "residuo_totale": "sum"
}).rename(columns={"Data": "giorni_pericolosi"}).reindex(anni_completi, fill_value=0)

summary["intensità_media"] = summary.apply(
    lambda row: row["residuo_totale"] / row["giorni_pericolosi"] if row["giorni_pericolosi"] > 0 else 0,
    axis=1
)

# ========== VISUALIZZAZIONE ==========
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(summary.index, summary["giorni_pericolosi"], color="orange", label="Giorni oltre soglia")
ax1.set_ylabel("Giorni oltre soglia", color="orange")
ax1.set_xlabel("Anno")
ax1.tick_params(axis='y', labelcolor="orange")

# Secondo asse y per intensità
ax2 = ax1.twinx()
ax2.plot(summary.index, summary["intensità_media"], color="blue", marker='o', label="Intensità media")
ax2.set_ylabel("Intensità cumulativa media", color="blue")
ax2.tick_params(axis='y', labelcolor="blue")

plt.title(f"Giorni pericolosi (≥ {min_soglie_superate} indici oltre soglia) - Metodo: {metodo_statistico}")
fig.tight_layout()
plt.grid(True)
plt.show()